<a href="https://colab.research.google.com/github/iremgngr/Machine_Learning/blob/main/Movie_Recommendation_tmbd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import gensim

from gensim.models import Word2Vec
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [73]:
df = pd.read_csv('tmdb_5000_movies.csv')
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [74]:
def genres_and_keywords_to_string(row):
    genres = json.loads(row['genres'])
    genres = ' '.join(''.join(j['name'].split()) for j in genres)
    keywords = json.loads(row['keywords'])
    keywords = ' '.join(''.join(j['name'].split()) for j in keywords)
    return "%s %s" % (genres, keywords)

In [75]:
df['string'] = df.apply(genres_and_keywords_to_string, axis=1)
sentences = [text.split() for text in df['string']]

movie2idx = pd.Series(df.index, index=df['title'])
movie2idx

,0
title,
Avatar,0
Pirates of the Caribbean: At World's End,1
Spectre,2
The Dark Knight Rises,3
John Carter,4
...,...
El Mariachi,4798
Newlyweds,4799
"Signed, Sealed, Delivered",4800


In [ ]:
# Word2Vec model
model_w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=1) # sg=0 or none 'cbow'

# By taking the average of all word vectors, we obtain a single vector belonging to the movie
def get_movie_vector_w2v(text):
    vectors_w2v = []
    for word in text.split():
        if word in model_w2v.wv:
            vectors_w2v.append(model_w2v.wv[word])
    return np.mean(vectors_w2v, axis=0) if vectors_w2v else np.zeros(100)

In [79]:
# GloVe model

# Load pre-trained GloVe embeddings
def load_glove_vectors(filepath):
    glove_vectors = {}
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            glove_vectors[word] = vector
    return glove_vectors

glove_vectors = load_glove_vectors("glove.6B.100d.txt")  # 100-D GloVe vectors

# Representing each movie with an average vector
def get_glove_vector(text):
    vectors_glove = []
    for word in text.split():
        if word in glove_vectors:
            vectors_glove.append(glove_vectors[word])
    return np.mean(vectors_glove, axis=0) if vectors_glove else np.zeros(100)

Recommendations for 'The Godfather' with GloVe:
867      The Godfather: Part III
3593    My Big Fat Greek Wedding
1915             The Conspirator
1047                  The Client
4474              The Conformist
Name: title, dtype: object


In [ ]:
# FastText model
model_fasttext = FastText(sentences, vector_size=100, window=5, min_count=1, sg=1)  # sg=1 'skip-gram'

# Representing each movie with an average vector
def get_fasttext_vector(text):
    vectors_fast = []
    for word in text.split():
        if word in model_fasttext.wv:
            vectors_fast.append(model_fasttext.wv[word])
    return np.mean(vectors_fast, axis=0) if vectors_fast else np.zeros(model_fasttext.vector_size)

In [92]:
#Store the vectors for each movie
df['vector_w2v'] = df['string'].apply(get_movie_vector_w2v)
df['vector_glove'] = df['string'].apply(get_glove_vector)
df['vector_fast'] = df['string'].apply(get_fasttext_vector)

df[['original_title', 'string', 'vector_w2v', 'vector_glove', 'vector_fast']].head()

,original_title,string,vector_w2v,vector_glove,vector_fast
0,Avatar,Action Adventure Fantasy ScienceFiction cultur...,"[0.0073092794, 0.100463346, 0.034303337, -0.01...","[-0.16781378, 0.37121814, 0.23992285, 0.067679...","[-0.5809489, 0.5956156, 0.12659065, 0.15037183..."
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drugabuse exoti...,"[0.0069271536, 0.09689112, 0.03412298, -0.0124...","[0.0968969, 0.12364302, 0.35212368, -0.2406627...","[-0.5653742, 0.55190885, 0.11827645, 0.1453362..."
2,Spectre,Action Adventure Crime spy basedonnovel secret...,"[0.015274191, 0.15983307, 0.054037474, -0.0246...","[0.10664, -0.48899332, 0.7798867, -0.30900332,...","[-0.526796, 0.6256143, 0.116251096, 0.14809941..."
3,The Dark Knight Rises,Action Crime Drama Thriller dccomics crimefigh...,"[0.0076489025, 0.08770591, 0.028540645, -0.011...","[0.07597284, -0.09402084, 0.12754847, -0.37413...","[-0.55825037, 0.57405907, 0.111256324, 0.14762..."
4,John Carter,Action Adventure ScienceFiction basedonnovel m...,"[0.01395386, 0.11908968, 0.041743267, -0.01830...","[-0.099363744, 0.073047355, 0.42189753, 0.2144...","[-0.56584984, 0.58180606, 0.11985258, 0.146492..."


In [ ]:
def recommend_movies_with_method(title, vector_column):
  # get the row in the dataframe for this movie
  idx = movie2idx[title]
  if type(idx) == pd.Series:
    idx = idx.iloc[0]

  # Convert the data to 2-D for cosine similarity to work correctly for each model
  query_vector = df[vector_column].iloc[idx].reshape(1, -1)
  all_vectors = np.vstack(df[vector_column].values)

  # calculate the pairwise similarities for this movie
  scores = cosine_similarity(query_vector, all_vectors).flatten()

  # get the indexes of the highest scoring movies
  recommended_idx = (-scores).argsort()[1:6]
  return df['title'].iloc[recommended_idx]

In [93]:
def recommend_movies(title):
    word2vec_recommendations = recommend_movies_with_method(title, 'vector_w2v')
    glove_recommendations = recommend_movies_with_method(title, 'vector_glove')
    fasttext_recommendations = recommend_movies_with_method(title, 'vector_fast')

    print(f"Recommendations based on Word2Vec for '{title}': \n{word2vec_recommendations}\n")
    print(f"Recommendations based on GloVe for '{title}': \n{glove_recommendations}\n")
    print(f"Recommendations based on FastText for '{title}': \n{fasttext_recommendations}\n\n")

#example
recommend_movies("Runaway Bride")
recommend_movies("Mission: Impossible III")
recommend_movies("The Godfather")

Recommendations based on Word2Vec for 'Runaway Bride': 
3837               She's the One
971              The Story of Us
2500                       Vamps
2554            The Wedding Date
1031    My Best Friend's Wedding
Name: title, dtype: object

Recommendations based on GloVe for 'Runaway Bride': 
2885       Married Life
971     The Story of Us
3262        Enough Said
1576         Bride Wars
593         The Dilemma
Name: title, dtype: object

Recommendations based on FastText for 'Runaway Bride': 
3925                  Whipped
3044              Get Over It
2554         The Wedding Date
272            Town & Country
1032    America's Sweethearts
Name: title, dtype: object


Recommendations based on Word2Vec for 'Mission: Impossible III': 
2156             Nancy Drew
873                   Shaft
1568                 Looper
4225               Repo Man
2581    Disturbing Behavior
Name: title, dtype: object

Recommendations based on GloVe for 'Mission: Impossible III': 
1077              